## imports

In [1]:
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import cv2
import numpy as np
import colorsys

# Part 1

## load dataset

In [2]:
def load_dataset():
  image_pickle_file_path = 'images.pkl'
  label_pickle_file_path = 'label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  # images = images.reshape(images.shape[0], -1)

  return images, labels

In [3]:
images, labels = load_dataset()
random_indices = np.random.choice(560, size=10, replace=False)
# Extract the randomly selected values
random_values = images[random_indices]

from dataclasses import dataclass, field


@dataclass
class Images:
    image: np.ndarray = None
    original_features = []
    features = []
    clusters = None




## Proccess on images
- Extract features
- normalize features
- make features a single dimnetion vector

In [4]:
def extract_features(image):
    features = []
    original_features = []

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # For each pixel in the image
    for i in range(img_hsv.shape[0]):
        for j in range(img_hsv.shape[1]):
            pixel = img_hsv[i, j]
            h, s, v = pixel
            x, y = i, j
            
            features.append((h, s, v, x, y))

    return features

In [5]:
def normalize_features(features):
    features_array = np.array(features)
    scaler = MinMaxScaler()
    normalized_features_array = scaler.fit_transform(features_array)
    normalized_features = normalized_features_array.tolist()
    return normalized_features

In [6]:
new_features = []

for img in random_values:
    features = extract_features(img)
    new_features.append(features)

In [7]:
# normalize features
for i, fimg in enumerate(new_features):
    new_features[i] = normalize_features(fimg)

In [8]:
def set_priority(arr, color, pos):
    color_columns = np.tile(arr[:, :3], (1, color))
    pos_columns = np.tile(arr[:, 3:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [9]:
new_features_freq = []
for nf in new_features:
    new_features_freq.append(set_priority(np.array(nf), 4, 2).tolist())

## clustering

### k-means

In [10]:
k_number = 15
kmeans = KMeans(n_clusters=k_number)
new_clusters = []
for nf in new_features_freq:
    clusters = kmeans.fit_predict(nf)
    new_clusters.append(clusters)

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\clus

In [11]:
for i, nc in enumerate(new_clusters):
    new_clusters[i] = nc.reshape(random_values.shape[1], random_values.shape[2])

## extracting clusters features for each image sepratedly

In [12]:
def find_matching_triads(array1, array2):   

    unique_values = np.unique(array1)
    result = []

    for value in unique_values:
        indexes = np.where(array1 == value)[0]
        triads = array2[indexes, :].tolist()
        result.append(triads)
        
    return result

### mean color for each cluster in each image

In [13]:
def calculate_mean(data):

    data_array = np.array(data)
    mean_values = np.mean(data_array, axis=0)
    mean_values = mean_values.astype(int)
    mean_values_list = mean_values.tolist()
    
    return mean_values_list

In [14]:
def mean_clusters_each_image(new_cls, random_values):

    all_images_mean_values = []
    
    for i, image_cls in enumerate(new_cls):

        one_image_clusters_mean_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_mean_values.append(calculate_mean(smp))

        all_images_mean_values.append(one_image_clusters_mean_values)

    return all_images_mean_values

In [15]:
mean_each_cluster_each_image = mean_clusters_each_image(new_clusters, random_values)

### variance color for each cluster in each image

In [16]:
def calculate_var(data):

    data_array = np.array(data)
    var_values = np.var(data_array, axis=0)
    var_values = var_values.astype(int)
    var_values_list = var_values.tolist()
    
    return var_values_list

In [17]:
def var_clusters_each_image(new_cls, random_values):

    all_images_var_values = []


    for i, image_cls in enumerate(new_cls):

        one_image_clusters_var_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_var_values.append(calculate_var(smp))

        all_images_var_values.append(one_image_clusters_var_values)

    return all_images_var_values

In [18]:
var_each_cluster_each_image = var_clusters_each_image(new_clusters, random_values)

### size of clusters in each image

In [19]:
def calculate_size(image_clusters):

    unique_clusters, counts = np.unique(image_clusters, return_counts=True)
    each_clusters_size = counts

    return each_clusters_size

In [20]:
size_each_cluster_each_image = []

for image_clusters in new_clusters:
    size_each_cluster_each_image.append(calculate_size(image_clusters))

### extract features faster and truelly

In [21]:
def extract_cluster_features(image_number, image_clusters, arr_new_features, k_number):
    clusters_features = []
    clusters = [[] for i in range(k_number)]
    for index, value in enumerate(image_clusters):
        clusters[value].append(arr_new_features[image_number, index])
    for cluster in clusters: # clusters contains all pixels which belong to the same cluster.
        temp_array = np.array(cluster)
        min_value = np.min(temp_array, axis=0)
        max_value = np.max(temp_array, axis=0)
        mean_value = np.mean(temp_array, axis=0)
        x_field = max_value[3] - min_value[3]
        y_field = max_value[4] - min_value[4]
        shape_field = np.array([x_field / y_field])
        combined_results = np.concatenate((min_value[:3], max_value[:3], mean_value[:3], shape_field), axis=None)
        clusters_features.append(combined_results)

    return np.array(clusters_features)

In [22]:
all_images_clusters_features = []
arr_new_features = np.array(new_features)
reshaped_image_clusters = image_clusters.reshape(image_clusters.shape[0] * image_clusters.shape[1])
for image_num, image_clusters in enumerate(new_clusters):
    all_images_clusters_features.append(extract_cluster_features(image_num, 
                                                                 reshaped_image_clusters,
                                                                 arr_new_features, k_number))

In [23]:
all_images_clusters_features

[array([[0.        , 0.        , 0.00796813, 0.97765363, 1.        ,
         0.93227092, 0.17137519, 0.47989524, 0.39964646, 1.07659574],
        [0.        , 0.        , 0.14741036, 1.        , 0.85098039,
         1.        , 0.30360943, 0.11084771, 0.50332374, 0.74907749],
        [0.        , 0.        , 0.14342629, 0.98882682, 0.61176471,
         0.68924303, 0.29668633, 0.08957777, 0.3969892 , 0.45045045],
        [0.        , 0.        , 0.        , 0.98882682, 1.        ,
         0.98007968, 0.15409808, 0.56292943, 0.45693345, 2.03433476],
        [0.        , 0.        , 0.11155378, 1.        , 1.        ,
         0.92031873, 0.14083361, 0.47696652, 0.55028485, 1.05761317],
        [0.        , 0.        , 0.06772908, 1.        , 1.        ,
         0.98007968, 0.18537871, 0.32248122, 0.43271552, 1.49640288],
        [0.        , 0.00784314, 0.1752988 , 0.99441341, 0.79607843,
         0.99601594, 0.11330863, 0.23379691, 0.41802549, 0.81428571],
        [0.        , 0.    

## display clustering

In [24]:
def display_clustering(image_clusters, each_cluster_color, image_index):
    # Create an empty array to hold the HSV values for each pixel
    rgb_image = np.zeros((image_clusters.shape[0], image_clusters.shape[1], 3), dtype=np.uint8)

    # Assign the corresponding HSV values to each pixel
    for i in range(image_clusters.shape[0]):
        for j in range(image_clusters.shape[1]):
            value = image_clusters[i, j] 
            hsv = each_cluster_color[value]
            # rgb = colorsys.hsv_to_rgb(hsv[0] / 360, hsv[1] / 100, hsv[2] / 100)
            # rgb_image[i, j] = np.round(np.array(rgb) * 255).astype(int)
            rgb_image[i, j] = hsv

    # Display the image with corresponding RGB values using cv2.imshow
    cv2.imshow('org', random_values[image_index])
    cv2.imshow('Image with Corresponding RGB Values', cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [25]:
each_cluster_color = [
    [255, 0, 0],    # Red
    [0, 255, 0],    # Green
    [0, 0, 255],    # Blue
    [255, 255, 0],  # Yellow
    [255, 0, 255],  # Magenta
    [0, 255, 255],  # Cyan
    [128, 0, 0],    # Maroon
    [0, 128, 0],    # Green (Dark)
    [0, 0, 128],    # Navy
    [128, 128, 128], # Gray
    [255, 128, 0],  # Orange
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 0],  # Olive
    [192, 192, 192]  # Silver
]

In [26]:
for i, img_clusters in enumerate(new_clusters):
    display_clustering(img_clusters, each_cluster_color, i)

# Part 2

## create clusters feature vectors 

## clustering clusters

In [27]:
kmeans_2 = KMeans(n_clusters=3)
clusters_features_flat = [np.concatenate([np.array(item[:-1]), item[-1]]) for item in clusters_features]
clusters_2 = kmeans.fit_predict(clusters_features_flat)
# clusters_2 = kmeans.fit_predict(clusters_features)
clusters_2

NameError: name 'clusters_features' is not defined

## Diaplay clustered cluster

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca = PCA(2)
data_transformed = pca.fit_transform(clusters_features_flat)

plt.scatter(data_transformed[:, 0], data_transformed[:, 1], c=clusters_2)
plt.show()
